# WLASL Training

The Word-Level American Sign Language (WLASL) dataset consists of approximately 12,000 videos of around 2,000 common words.

For training, we use the LSTM neural network for sequence classification
and TensorFlow/Keras for model training.

## Set-Up

### Install Dependencies

In [ ]:
import os
import sys
import subprocess

packages = [
    "numpy==1.26.4",
    "protobuf==4.25.3",
    "mediapipe==0.10.21",
    "opencv-python-headless==4.8.1.78",
    "scikit-learn==1.3.2",
    "matplotlib"
]

print("Installing dependencies...")
print("="*60)

In [ ]:
import sys
import mediapipe as mp
import cv2
import sklearn
import tensorflow as tf
import numpy as np

print("="*60)
print("Verifying installations...")
print(f"  Python version: {sys.version.split()[0]}")
print(f"  TensorFlow: {tf.__version__}")
print(f"  MediaPipe: {mp.__version__}")
print(f"  OpenCV: {cv2.__version__}")
print(f"  Scikit-learn: {sklearn.__version__}")
print(f"  NumPy: {np.__version__}") # Should be 1.26.4

print("="*60)

### Import Libraries

In [ ]:
# Import all required libraries
print("Importing libraries...")

# Standard library imports
import json
import os
import sys
from collections import Counter
from typing import List, Sequence

# Third-party imports
try:
    import cv2
    print("OpenCV imported")
except ImportError as e:
    print(f"OpenCV import failed: {e}")
    sys.exit(1)

try:
    import mediapipe as mp
    from mediapipe.tasks import python as mp_python
    from mediapipe.tasks.python import vision
    print("MediaPipe imported")
except ImportError as e:
    print(f"MediaPipe import failed: {e}")
    print("  Run the installation cell again.")
    sys.exit(1)

try:
    import numpy as np
    print("NumPy imported")
except ImportError as e:
    print(f"NumPy import failed: {e}")
    sys.exit(1)

try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    print("TensorFlow/Keras imported")
except ImportError as e:
    print(f"TensorFlow import failed: {e}")
    sys.exit(1)

try:
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    print("Scikit-learn imported")
except ImportError as e:
    print(f"Scikit-learn import failed: {e}")
    sys.exit(1)

# Constants
NUM_HANDS = 2
NUM_LANDMARKS = 21
COORDS = 3
FEATURE_VECTOR_LEN = NUM_HANDS * NUM_LANDMARKS * COORDS
WRIST_IDX = 0
MIDDLE_MCP_IDX = 9

print("\n" + "="*60)
print("All libraries imported successfully!")
print("="*60)

### Mount Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted successfully!")

### Configuration

In [ ]:
# File paths
NPZ_PATH = "wlasl_landmarks.npz"  # Or use Drive path: "/content/drive/MyDrive/WLASL Dataset/wlasl_landmarks.npz"
LABELS_OUTPUT = "wlasl_labels.npy"
MODEL_OUTPUT = "wlasl_sequence_model.keras"

# Training parameters
TEST_SIZE = 0.2
EPOCHS = 80
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
LSTM_UNITS = [128, 64]
DENSE_UNITS = 64
DROPOUT = 0.5
PATIENCE = 10                 # Early stopping patience

print("Configuration loaded")
print(f"\nLooking for preprocessed data at: {NPZ_PATH}")

### Build LSTM Model

In [ ]:
def build_model(
    input_shape: tuple,
    num_classes: int,
    lstm_units: list,
    dense_units: int,
    dropout: float,
    learning_rate: float,
) -> keras.Model:
    """
    Build LSTM model for sequence classification.

    Architecture:
    - Masking layer (ignores padded zeros)
    - LSTM layer 1 (with return_sequences=True)
    - Dropout
    - LSTM layer 2
    - Dropout
    - Dense layer (ReLU)
    - Output layer (Softmax)
    """
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Masking(mask_value=0.0),  # Ignore padded frames
        layers.LSTM(lstm_units[0], return_sequences=True, use_cudnn=False), # Disable cuDNN
        layers.Dropout(dropout),
        layers.LSTM(lstm_units[1], use_cudnn=False), # Disable cuDNN
        layers.Dropout(dropout),
        layers.Dense(dense_units, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

print("Model builder function defined")

### Train the Model

In [ ]:
# Set random seed for reproducibility
keras.utils.set_random_seed(42)
try:
    tf.config.experimental.enable_op_determinism()
except Exception:
    pass

# Load preprocessed data
print("Loading preprocessed data...")

data = np.load(NPZ_PATH)
sequences = data["sequences"].astype(np.float32)
labels = data["labels"]

print(f"Loaded {sequences.shape[0]} sequences")
print(f"  Shape: {sequences.shape}")

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
np.save(LABELS_OUTPUT, label_encoder.classes_)
print(f"Saved label order to {LABELS_OUTPUT} ({len(label_encoder.classes_)} classes)")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    sequences,
    encoded_labels,
    test_size=TEST_SIZE,
    random_state=42,
    stratify=encoded_labels,
)

print(f"\nTrain samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

# Build model
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = len(label_encoder.classes_)

print(f"\nBuilding model...")
print(f"  Input shape: {input_shape}")
print(f"  Number of classes: {num_classes}")

model = build_model(
    input_shape=input_shape,
    num_classes=num_classes,
    lstm_units=LSTM_UNITS,
    dense_units=DENSE_UNITS,
    dropout=DROPOUT,
    learning_rate=LEARNING_RATE,
)

model.summary()

# Setup callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        MODEL_OUTPUT,
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    ),
]

# Train model
print(f"\nStarting training for up to {EPOCHS} epochs...")
history = model.fit(
    X_train,
    y_train,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1,
)

print("\nTraining complete!")

### Evaluate Model Performance

In [ ]:
# Generate predictions
print("Generating predictions on test set...")
predictions = model.predict(X_test, verbose=0)
y_pred = np.argmax(predictions, axis=1)

# Calculate accuracy
acc = accuracy_score(y_test, y_pred)
print(f"\n{'='*60}")
print(f"TEST ACCURACY: {acc:.4f} ({acc*100:.2f}%)")
print(f"{'='*60}")

# Classification report
print("\nClassification Report:")
print("="*60)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Confusion matrix
print("\nConfusion Matrix:")
print("="*60)
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Save final model (ensure it's saved even if early stopping didn't trigger)
model.save(MODEL_OUTPUT)
print(f"\nModel saved to {MODEL_OUTPUT}")
print(f"Labels saved to {LABELS_OUTPUT}")

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)
print(f"\nTo use this model:")
print(f"1. Download '{MODEL_OUTPUT}' and '{LABELS_OUTPUT}'")
print(f"2. Load them in your inference script")
print(f"3. Process videos the same way (landmarks -> normalize -> predict)")

### Plot Training History

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()